**ids-pdl09-tut.ipynb**: This Jupyter notebook is provided by Joachim Vogt for the *Python Data Lab* of the module *CH-700 Introduction to Data Science* offered in Fall 2023 at Constructor University. Jupyter notebooks and other learning resources are available from a dedicated *module platform*.

# Working with Pandas

This tutorial explores selected functionality provided by the Pandas package for transforming and processing tabular data. It builds on the companion tutorial *Pandas basics*. Follow the instructions below to learn to

- [ ] stack and unstack Pandas DataFrame objects,
- [ ] define Pandas MultiIndex objects for hierarchical indexing,
- [ ] apply GroupBy methods for data filtering and aggregation,
- [ ] construct pivot tables of DataFrame objects.

If you wish to keep track of your progress, you may edit this markdown cell, check a box in the list above after having worked through the respective part of this notebook, and save the file.

*Short exercises* are embedded in this notebook. *Sample solutions* can be found at the end of the document.

## Preparation

The following data file is expected to reside in the working directory. Identify the file on the module platform and upload it to the same folder as this Jupyter notebook.

- `life-expectancy-at-birth-total-years.csv`: Life expectancy at birth 1960-2019, published by the [World Bank, 2021-07-30](http://data.worldbank.org/data-catalog/world-development-indicators), available from [Our World in Data](https://ourworldindata.org/grapher/life-expectancy-at-birth-total-years).

Run the following code cell to import standard Python data science libraries. The NumPy module facilitates efficient processing of numerical arrays, and is usually imported as `np`. From the matplotlib library we import the package `pyplot` using the standard abbreviation `plt`. The magic command `%matplotlib inline` (IPython shell) allows for inline display of graphics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Importing Pandas

It is common practice to import the Pandas library using the prefix `pd`.

In [ ]:
import pandas as pd

### Pandas documentation and tutorials

Pandas is hosted at [https://pandas.pydata.org](https://pandas.pydata.org), with extensive [documentation](https://pandas.pydata.org/docs) and [tutorials](https://pandas.pydata.org/docs/getting_started/intro_tutorials/). If you are familiar with other tools for analyses of tabular data (R, SQL, spreadsheets, SAS, Stata), it may be worthwhile comparing terminologies, see the page [Comparisons with other tools](https://pandas.pydata.org/docs/getting_started/comparisons/).

Note the official spelling is *pandas* (lower case). In this tutorial the package name is capitalized (*Pandas*) to avoid confusion with a bunch of bears.

## Revisiting the life expectancy data set

In the companion tutorial notebook *Introduction to Pandas*, life expectancy at birth data from the file `life-expectancy-at-birth-total-years.csv` were loaded, rearranged, and processed to yield a tabular representation with rows containing the data for individual years, and columns containing the data for individual countries. The procedure is repeated in the code cell below. In the exercises of the same notebook, a similar tabular array was constructed, with yearly data forming the columns, and country data forming the rows. DataFrame objects in such formats allowed for convenient computation of summary statistics, as a particular example of an *aggregation* operation.

Recall that the procedure was not considered computationally efficient, but only meant to illustrate the methods introduced in the context of the companion tutorial.

In [ ]:
###  Store all data from the file in a DataFrame.
leb_full = pd.read_csv('life-expectancy-at-birth-total-years.csv')
### Initialize new DataFrame with the data from Australia.
ind_aus = leb_full['Code']=='AUS'
leb_6cnt = pd.DataFrame({'Year':leb_full[ind_aus].iloc[:,2].values,
                    'Life exp. (AUS)':leb_full[ind_aus].iloc[:,3].values})
### Successively add new columns with data for five countries.
Codes = ['BRA','CHN','FRA','NGA','USA']
for code in Codes:
    ind = leb_full['Code']==code
    leb_6cnt = leb_6cnt.merge(pd.DataFrame({'Year':leb_full[ind].iloc[:,2].values,
                                'Life exp. ('+code+')':leb_full[ind].iloc[:,3].values}))
### Display the final result.
display(leb_6cnt.head())

The present tutorial is concerned with more practical methods to yield such rearrangements of tabular data, and the application of aggregation operation. From the full DataFrame `leb_full` we remove rows for entities that are not single countries but world regions. Since in most such cases the Code is empty (`''`) except for the world as a whole (entity `'World'`), these removals are effectively accomplished as follows.

In [ ]:
print('Shape of leb_full before applying dropna()      : ',leb_full.shape)
leb_full.dropna(inplace=True)
print('Shape of leb_full after  applying dropna()      : ',leb_full.shape)
leb_full = leb_full[leb_full['Entity']!='World']
print("Shape of leb_full after ignoring entity 'World' : ",leb_full.shape)

The life expectancy column label is inconveniently long, so we rename.

In [ ]:
leb_full.rename(columns={leb_full.columns[3]:'Life exp.'},inplace=True,errors='raise')
display(leb_full)

## Hierarchical indexing

A DataFrame can be conveniently restructured using *hierarchical indexing*. The underlying concept is illustrated using a smaller data set considered already in the companion tutorial *Introduction to Pandas*, , namely, the number of residents of four German cities in the years 1970, 1990, 2010, according to [Wikipedia (accessed on 2022-07-26)](https://de.wikipedia.org/wiki/Liste_der_Gro%C3%9F-_und_Mittelst%C3%A4dte_in_Deutschland).

 City      | 1970   | 1990   | 2010   | 
:----------|:------:|:------:|:------:|
 Bremen    | 592533 | 551219 | 547340 |
 Dresden   | 502432 | 490571 | 523058 |
 Essen     | 696419 | 626973 | 574635 |
 Stuttgart | 634202 | 579988 | 606588 |

### MultiIndex objects

In the cell below, a DataFrame is constructed and displayed.

In [ ]:
res1 = pd.DataFrame( {'1970':[592533,502432,696419,634202],
                      '1990':[551219,490571,626973,579988],
                      '2010':[547340,523058,574635,606588]},
                    index=['Bremen','Dresden','Essen','Stuttgart'])
display(res1)

The index consists of the four city names.

In [ ]:
print(res1.index)

Now let us apply the Pandas function `stack()` and see what we obtain.

In [ ]:
res2 = res1.stack()
display(res2)

The result `res2` is a Pandas Series. The four rows of `res1` have been stacked to form a one-dimensional array, i.e., a single column vector of data. As revealed by `res2.index`, the index has become a set of tuples, a so-called `MultiIndex` object.

In [ ]:
print(res2.index)

This representation of the data is close to the structure of a data file such as `life-expectancy-at-birth-total-years.csv`. Further below we show how a MultiIndex object can be constructed from its columns.

The `stack()` operation is reversed through `unstack()`. The resulting object `res3` is a DataFrame with the same format and the same content as the initial DataFrame `res1`.

In [ ]:
res3 = res2.unstack()
display(res3)
print(res3.index)

MultiIndex objects can be defined in several ways. If individual tuple elements are separate columns of a data file, they can be stored in arrays which in turn yields a MultiIndex object through the `from_arrays()` method. Together with the column vector of data from the same file, a Series object just as `res2` can then be constructed from the MultiIndex.

The `from_arrays()` approach to MultIndex definition is demonstrated in the code cell below.

In [ ]:
cities = ['Bremen', 'Bremen', 'Bremen', 'Dresden', 'Dresden', 'Dresden', 
          'Essen', 'Essen', 'Essen', 'Stuttgart', 'Stuttgart', 'Stuttgart']
print('Array of city names:')
print(cities)
years = [1970, 1990, 2010, 1970, 1990, 2010, 
         1970, 1990, 2010, 1970, 1990, 2010]
print('\nArray of years:')
print(years)
mulind4 = pd.MultiIndex.from_arrays([cities,years])
print('\nMultiIndex constructed from the two arrays:')
print(mulind4)

With the resident numbers provided as a single vector, a suitable Series object is created using the MultiIndex.

In [ ]:
residents = [592533,551219,547340,502432,490571,523058,
             696419,626973,574635,634202,579988,606588]
print('Array of resident numbers:')
print(residents)
res4 = pd.Series(residents,index=mulind4)
print('\nSeries constructed from MultiIndex and array of resident numbers:')
print(res4)
print('\nUnstacking the Series produces the following DataFrame:')
display(res4.unstack())

For further options of MultiIndex construction, see the documentation of the MultiIndex methods `from_tuples()` and `from_product()`.

### Exercise: Hierarchical indexing

Consult the Pandas documentation to understand the logic and the syntax of the MultiIndex method `from_product()`. Construct a MultiIndex object similar to `res4` in the German city residents example above, then create a Series object that is unstacked to obtain a DataFrame of the same format as `res1`. 

In [ ]:
### Array of resident numbers.
residents = [592533,551219,547340,502432,490571,523058,
             696419,626973,574635,634202,579988,606588]
### MultiIndex constructed from product.

### Series constructed from MultiIndex.

### Unstacking the Series produces a DataFrame.


### Hierarchical indexing applied to life expectancy data 

The `MultiIndex.from_arrays()` approach is applied to the life expectancy data stored in the DataFrame `leb_full`.

In [ ]:
display(leb_full)

Store the column of life expectancy values.

In [ ]:
life_exp_values = leb_full.iloc[:,3].values
print(life_exp_values)

Construct the MultiIndex object from the `'Code'` and `'Year'` columns.

In [ ]:
codes = leb_full['Code'].values
years = leb_full['Year'].values
mulind = pd.MultiIndex.from_arrays([codes,years])
print(mulind)

Using the array of life expectancy values with the MultiIndex object as an index, we construct a Series object that is then unstacked to yield a DataFrame in the desired format.

In [ ]:
leb = pd.Series(life_exp_values,index=mulind).unstack()
display(leb.head())

Rows and columns are swapped through the application of `transpose()`.

In [ ]:
display(leb.transpose().head())

The `describe()` method yields summary statistics of a DataFrame.

In [ ]:
display(leb.describe().transpose().head())

## GroupBy mechanism

The summary statistics computed from the restructured life expectancy DataFrame `leb` are examples of aggregates. If only such results of aggregation operations or related functions are sought, one may invoke the Pandas GroupBy mechanism with the original DataFrame `leb_full`, thus effectively skipping the potentially resource-consuming restructuring step.

The GroupBy mechanism is usually understood as a sequence of three operations.
1. *Split* a DataFrame according to the values of a specific key to produced groups.
2. *Apply* a function (aggregation, transformation, filtering, ...) within each group. 
3. *Combine* the results into an array.

As an example, the following line produces life expectancy summary statistics directly from the original DataFrame `leb_full`.

- `'Year'` is the key, defining yearly groups of the countries contributing data.
- `'Life exp.'` indicates which column is supposed to be aggregated or, more generally, processed by the applied function.
- `describe()`: is the operation applied to the data.

In [ ]:
leb_full.groupby('Year')['Life exp.'].describe().head()

If a particular statistic is of interest, replace `describe()` by the appropriate function.

In [ ]:
leb_full.groupby('Year')['Life exp.'].quantile(0.75).head()

User-defined subsets of summary statistics are display using `aggregate()`.

In [ ]:
leb_full.groupby('Year')['Life exp.'].aggregate(['min','mean','max']).head()

More general functions can be incorporated into the split-apply-combine scheme by means of the `apply()` method. The function defined in the following cell computes the interquartile range of an array.

In [ ]:
def iqr(x): return np.quantile(x,0.75)-np.quantile(x,0.25)
leb_full.groupby('Year')['Life exp.'].apply(iqr).head()

### Exercise: GroupBy mechanism

The `groupby()` function accepts not only column labels as keys but also lists, dictionaries, NumPy arrays, Pandas series, and user-defined functions. See the Pandas documentation for further information on the logic and the syntax of the function `groupby()`. Armed with this information, aggregate the life expectancy data from the DataFrame `leb_full` in decades, using `min`, `mean`, and `max` as aggregation functions.

*Hint*: Consult [section 3.08 Aggregation and Grouping](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html) of the 
[Python Data Science Handbook by Jake Vanderplas](https://jakevdp.github.io/PythonDataScienceHandbook/) for an effective implementation of decade aggregation.

In [ ]:
### Construct the variable decade as in section 3.08 of the PDSH by Jake Vanderplas.

### Apply groupby() to obtain decadal aggregation of min, mean, max.


## Pivot tables

Following [section 3.09 Pivot Tables](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) of the [Python Data Science Handbook by Jake Vanderplas](https://jakevdp.github.io/PythonDataScienceHandbook/), we introduce pivot tables as an extension of the GroupBy logic from Pandas Series to DataFrame objects using a dataset of Titanic passengers, available through the Python package Seaborn (requires internet access).

In [ ]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
display(titanic)
### Uncommenting the next line stores the data in a csv file (for offline work).
#titanic.to_csv('titanic.csv')
### Uncommenting the next line loads the data from the local csv file 'titanic.csv'.
#titanic = pd.read_csv('titanic.csv')

Applying the GroupBy syntax to the `survived` column with binary values `1` and `0`, using `sex` and `class` as keys and `mean` as the aggregation function, produces the following table of survival rates.

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

Since this kind of operation on tabular data is very common, it is customized by means of the Pandas function `pivot_table()`.

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', aggfunc='mean')

### Exercise: Pivot tables

See the Pandas documentation for further keyword arguments of `pivot_table()`. Study the application to birthrate data in [section 3.09 Pivot Tables](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html) of the [Python Data Science Handbook by Jake Vanderplas](https://jakevdp.github.io/PythonDataScienceHandbook/).

---
---

## Solutions

### Solution: Hierarchical indexing

In [ ]:
### Array of resident numbers.
residents = [592533,551219,547340,502432,490571,523058,
             696419,626973,574635,634202,579988,606588]
### MultiIndex constructed from product.
mulind5 = pd.MultiIndex.from_product([['Bremen','Dresden','Essen','Stuttgart'],
                                      [1970,1990,2010]])
print('MultiIndex constructed from product:')
print(mulind5)
### Series constructed from MultiIndex.
res5 = pd.Series(residents,index=mulind5)
print('\nSeries constructed from MultiIndex and array of resident numbers:')
print(res5)
### Unstacking the Series produces a DataFrame.
print('\nUnstacking the Series produces the following DataFrame:')
display(res5.unstack())

### Solution: GroupBy mechanism

In [ ]:
### Construct the variable decade as in section 3.08 of the PDSH by Jake Vanderplas.
decade = 10*( leb_full['Year'] // 10 )
decade = decade.astype(str)+'s'
decade.name = 'Decade'
### Apply groupby() to obtain decadal aggregation of min, mean, max.
leb_full.groupby(decade)['Life exp.'].aggregate(['min','mean','max'])

---
---